In [12]:
#Watershed Algo

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image


img = cv.imread('6122-07696135en_Masterfile.jpg', cv.IMREAD_ANYCOLOR)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
print(ret)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]


cv.imshow("pic", img)
cv.waitKey(0)
cv.destroyAllWindows()


128.0


In [13]:
#Dilation and Erosion


# Python program to demonstrate erosion and
# dilation of images.
import cv2
import numpy as np

# Reading the input image
img = cv2.imread('Satellite_photo_of_old_Lot_54.jpg', 0)

# Taking a matrix of size 5 as the kernel
kernel = np.ones((5, 5), np.uint8)

# The first parameter is the original image,
# kernel is the matrix with which image is
# convolved and third parameter is the number
# of iterations, which will determine how much
# you want to erode/dilate a given image.
img_erosion = cv2.erode(img, kernel, iterations=1)
img_dilation = cv2.dilate(img, kernel, iterations=1)

# cv2.imshow('Input', img)
# cv2.imshow('Erosion', img_erosion)
# cv2.imshow('Dilation', img_dilation)

cv2.imshow("Parking Lots", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Saving the image
cv2.imwrite('ToContour.png', img_erosion)
  


True

In [14]:
#Contouring

import cv2
import numpy as np

# Let's load a simple image with 3 black squares
image = cv2.imread('ToContour.png')
cv2.waitKey(0)

# Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.waitKey(0)

# Finding Contours
# Use a copy of the image e.g. edged.copy()
# since findContours alters the image
contours, hierarchy = cv2.findContours(edged,
	cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Draw all contours
# -1 signifies drawing all contours
cv2.drawContours(image, contours, -1, (0, 255, 0), 3)

cv2.imshow('Contours', image)
cv2.imwrite('Contoured.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of Contours found = 238


In [17]:
#Experiment 1

import numpy as np
import cv2

# Load the aerial image
img = cv2.imread("aerial-view-bird-s-eye-view-cars-design.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the grayscale image to create a binary image
_, binary = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

# Use morphological operations to remove noise and fill in small gaps in the image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# Use contour detection to find the contours of the parking lot
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate over the contours and draw rectangles around the parking lots
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show the original image with the parking lots highlighted
cv2.imshow("Parking Lots", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
#Experiment 2

import numpy as np
import cv2

# Load the image
img = cv2.imread('Satellite_photo_of_old_Lot_54.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply a Gaussian blur to the image to reduce noise
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply Canny edge detection to the image
edges = cv2.Canny(gray, 50, 150)

# Create a kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# Apply a closing operation to close gaps in the edges
closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# Create a blank mask to hold the road regions
mask = np.zeros_like(closed)

# Define the ROI (region of interest) for the roads
im2, contours, = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through the contours and fill in the mask for the road regions
for c in contours:
    if cv2.contourArea(c) > 5000:
        cv2.drawContours(mask, [c], -1, (255, 255, 255), -1)

# Apply the mask to the original image to extract the roads
roads = cv2.bitwise_and(img, img, mask=mask)

# Save the resulting image
cv2.imwrite('roads.jpg', roads)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::contourArea'


In [ ]:
#Video Color FILTER

import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while(1):
	_, frame = cap.read()
	# It converts the BGR color space of image to HSV color space
	hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	
	# Threshold of blue in HSV space
	lower_blue = np.array([60, 35, 140])
	upper_blue = np.array([180, 255, 255])

	# preparing the mask to overlay
	mask = cv2.inRange(hsv, lower_blue, upper_blue)
	
	# The black region in the mask has the value of 0,
	# so when multiplied with original image removes all non-blue regions
	result = cv2.bitwise_and(frame, frame, mask = mask)

	cv2.imshow('frame', frame)
	cv2.imshow('mask', mask)
	cv2.imshow('result', result)
	
	cv2.waitKey(0)

cv2.destroyAllWindows()
cap.release()


In [9]:
import cv2 as cv
import numpy as np

img=cv.imread('6122-07696135en_Masterfile.jpg')
hsv=cv.cvtColor(img,cv.COLOR_BGR2HSV)
lower_white=np.array([0,0,200])
upper_white=np.array([255,55,255])

masked=cv.inRange(hsv,lower_white,upper_white)
res=cv.bitwise_and(img,img,mask=masked)
greyres=cv.cvtColor(res,cv.COLOR_BGR2GRAY)


blur=cv.GaussianBlur(img,(5,5),0)
sigma = np.std(blur)
mean = np.mean(blur)
lower = int(max(0, (mean - sigma)))
upper = int(min(255, (mean + sigma)))
canny=cv.Canny(blur,lower,upper)

rectangle=cv.rectangle(canny,(0,0),(160,48),(0,0,0),-1)
mask=cv.bitwise_and(rectangle,canny)

thresh=cv.threshold(grey,180,255,cv.THRESH_BINARY)[1]
kernel=cv.getStructuringElement(cv.MORPH_RECT,(5,5))
dilate=cv.morphologyEx(thresh,cv.MORPH_DILATE,kernel)
diff=cv.absdiff(dilate,thresh)
edges=255-diff


blur=cv.GaussianBlur(mask,(3,3),0)
canny=cv.Canny(blur,85,255)



grey=cv.cvtColor(img,cv.COLOR_BGR2GRAY) 
imgHLS=cv.cvtColor(img,cv.COLOR_BGR2HLS) 
Lchannel=imgHLS[:,:,1]
maskHLS=cv.inRange(Lchannel,200,255)
resHLS=cv.bitwise_and(img,img,mask=maskHLS)

rectangle=cv.rectangle(grey,(0,0),(160,48),(0,0,0),-1)
mask=cv.bitwise_and(rectangle,grey)
no=cv.bitwise_not(rectangle)

v=np.median(no)
sigma=0.33
lower=int(max(0,(1.0-sigma)*v))
upper=int(min(255,(1.0+sigma)*v))


greyHLS=cv.cvtColor(resHLS,cv.COLOR_BGR2GRAY)
kernel=cv.getStructuringElement(cv.MORPH_RECT,(5,5))#
canny=cv.Canny(mask,lower,upper)#
dilate=cv.dilate(canny,kernel, iterations=1)#

mask2=cv.bitwise_and(dilate,greyHLS)#


cv.imshow('result',mask2)

cv.waitKey(0)

-1